In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reward Model

In [2]:
!pip install transformers==4.40.2
!pip install datasets==2.19.1
!pip install accelerate==0.30.1
!pip install trl==0.8.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 27.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.1
    Uninstalling transformers-4.41.1:
      Successfully uninstalled transformers-4.41.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x8

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
rm_tokenizer = AutoTokenizer.from_pretrained("./drive/MyDrive/Colab/T5-jeju/data/saved_models/roberta_rm")
rm_model = AutoModelForSequenceClassification.from_pretrained("./drive/MyDrive/Colab/T5-jeju/data/saved_models/roberta_rm", num_labels=2).to(device)

PPO

In [6]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl.core import LengthSampler
import multiprocessing

In [7]:
#####
file_path = '/content/drive/My Drive/Colab/T5-jeju/data/filtered_train_ppo_2.tsv'

train_df = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip')

num_train = 50000
num_valid = 1000
sliced_train_df = train_df.iloc[:num_train]
sliced_valid_df = train_df.iloc[num_train:num_train+num_valid]
sliced_train_df.to_csv("sliced_train.tsv", sep='\t', index=False)
sliced_valid_df.to_csv("sliced_valid.tsv", sep='\t', index=False)

data_files = {"train": "sliced_train.tsv", "valid": "sliced_valid.tsv"}
train_dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

In [8]:
max_token_length = 64
NUM_CPU = multiprocessing.cpu_count()

In [9]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['dialect_form'],
                             text_target=examples['standard_form'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

In [10]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab/T5-jeju/data/saved_models/mixed_plus_multi/results")

# tokenizer.pad_token = tokenizer.eos_token

tokenized_datasets = train_dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서
                                 # 아래서 콜레이터가 패딩을 못해서 에러남
                                 remove_columns=train_dataset["train"].column_names,
                                 num_proc=NUM_CPU)
###

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})


In [12]:
model_name = "/content/drive/MyDrive/Colab/T5-jeju/data/saved_models/mixed_plus_multi/results"
config = PPOConfig(
    model_name=model_name,
    learning_rate=1.41e-5,
    batch_size=128,
    mini_batch_size=128,
    ppo_epochs=1
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [13]:
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name).to(device)
ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name).to(device)
# tokenizer = AutoTokenizer.from_pretrained(config.model_name)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
sen = ["집 앞에 손 매는 건 집에 아픈 사람이 있거나 굿을 하거나 다 이유가 있지", "집 앞에 손 매는 것은 집에 아픈 사람이 있거나 굿을 하거나 다 이유가 있지"]
tok = rm_tokenizer(sen, padding=True, truncation=True, return_tensors="pt")
tok = {k: v.to(device) for k, v in tok.items()}
output = rm_model(**tok).logits
output2 = [torch.tensor([i]) for i in output[:, 1]]
print(sen[0])
print(output2[0])
print(sen[1])
print(output2[1])


집 앞에 손 매는 건 집에 아픈 사람이 있거나 굿을 하거나 다 이유가 있지
tensor([3.4973])
집 앞에 손 매는 것은 집에 아픈 사람이 있거나 굿을 하거나 다 이유가 있지
tensor([3.5925])


In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=tokenized_datasets['train'], data_collator=data_collator)

In [ ]:
import re
import torch

In [ ]:
sen = ["안녕하십니까까까까", "녕안하하하하하하하하핳하하하하하하하하하하하", "이하하"]
tok = rm_tokenizer(sen, padding=True, truncation=True, return_tensors="pt")
tok = {k: v.to(device) for k, v in tok.items()}
output = rm_model(**tok).logits
output2 = [torch.tensor([i]) for i in output[:, 1]]
print(output)
print(output2)

tensor([[-1.9446,  1.8862],
        [ 1.4886, -1.8402],
        [ 0.4976, -0.8518]], device='cuda:0', grad_fn=<AddmmBackward0>)
[tensor([1.8862]), tensor([-1.8402]), tensor([-0.8518])]


In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [ ]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


pattern = r'<pad>|</s>|<unk>'

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), total=len(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        # response = ppo_trainer.generate(query, **generation_kwargs)
        response = ppo_trainer.generate(query, max_length=max_token_length, num_beams=5)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [re.sub(pattern, '', tokenizer.decode(r.squeeze(), skip_special_tokens=True)) for r in response_tensors]
    # print(batch["response"])
    tok = rm_tokenizer(batch["response"], padding=True, truncation=True, return_tensors="pt")
    tok = {k: v.to(device) for k, v in tok.items()}
    with torch.no_grad():
      output = rm_model(**tok).logits
    # print(output)
    rewards = [torch.tensor([i]) for i in output[:, 1]]
    # print(rewards)

    temp_q = list(torch.unbind(query_tensors))
    q_t = [q for q in temp_q]

    # print("q_t: ", query_tensors)
    # print("q_t22: ", q_t)
    # print("r_t: ", response_tensors)
    # print("r: ", rewards)
    stats = ppo_trainer.step(q_t, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 1/390 [00:31<3:24:22, 31.52s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -1.06 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
  3%|▎         | 10/390 [05:18<3:18:43, 31.38s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -1.36 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
  3%|▎         | 11/390 [05:54<3:27:29, 32.85s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1289: UserWarnin

In [ ]:
ppo_trainer.save_pretrained("./drive/MyDrive/Colab/T5-jeju/data/saved_models/rlhf_multi4")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1407: UserWarning: Cannot retrieve user information assuming you are running in offline mode.
  warnings.warn("Cannot retrieve user information assuming you are running in offline mode.")


In [ ]:
torch.cuda.empty_cache()